### Cardiac Arrhythmia Multy-Class Classification 

Analyze data and address missing data if there is any. 

Decide aboute a good evaluation strategy and justify your choice. 

Find the best parameters for the following classification models: 
- KNN classifcation 
- Logistic Regression
- Linear Supprt Vector Machine
- Kerenilzed Support Vector Machine
- Decision Tree
- Random Forest 

Then use different bagging and boosting methods to boost the results? Do you see any significant change? Why or why not? 

Next, use data reduction method you have learned in class to reduce the size of data, and agian try above models. Do you get better results? Justify your answer. 

<font color = 'red'>Due date for full credit: April 4, 11:59 PM
    <br>
    Due date for partial credit: April 6, 11:59 PM.
    <br> No submission will be accepted after April 6. 
    <br> Please note that your term paper is also due April 6. 
    </font>

In [3]:
import pandas as pd
data = pd.read_csv('cardiac_arrhythmia.csv',header=None,na_values=['?'])

In [4]:
data.shape

(452, 280)

### Creating a function to determine the number of Missing values in every column 

In [73]:
#Create a new function:
def num_missing(x):
    return sum(x.isnull())

#Applying per column:
print("\nMissing values per column:")
print(data.apply(num_missing, axis=0)) #axis=0 defines that function is to be applied on each column


Missing values per column:
0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       8
11      22
12       1
13     376
14       1
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
      ... 
250      0
251      0
252      0
253      0
254      0
255      0
256      0
257      0
258      0
259      0
260      0
261      0
262      0
263      0
264      0
265      0
266      0
267      0
268      0
269      0
270      0
271      0
272      0
273      0
274      0
275      0
276      0
277      0
278      0
279      0
Length: 280, dtype: int64


### Observed that there are some columns with missing Values. Deleting those columns from the data

In [5]:
data.dropna(axis=1,how ='any',inplace = True)

In [6]:
data.shape

(452, 275)

### This deleted 5 columns from dataset. Next we look if there are any single valued columns (same value for all patient records)

In [76]:
cols = list(data)
nunique = data.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
data.drop(cols_to_drop, axis=1,inplace = True)

### This deleted around 13 columns from the data set 

In [77]:
data.shape

(452, 258)

### KNN Classification - Finding best params using grid serach and computing cross val scores

In [102]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [115]:
X_features = data.iloc[:,0:257]
y_response = data.iloc[:,257]

In [116]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y_response, random_state=0)

Scaling of data

In [165]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [166]:
knn = KNeighborsClassifier()

Grid Search to find the best K value for KNN classification

In [178]:
k = np.arange(5)+1
param_knn= {'n_neighbors':k}
clf = GridSearchCV(knn, param_knn, cv=5)
clf.fit(X_train_scaled,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=5, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': array([1, 2, 3, 4, 5])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

Best Parameters from Grid Search

In [179]:
clf.best_params_

{'n_neighbors': 3}

Training and test data accuracies

In [180]:
print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(clf.score(X_train_scaled, y_train)))
print('Accuracy of KNN classifier on test set: {:.2f}'
     .format(clf.score(X_test_scaled, y_test)))

Accuracy of KNN classifier on training set: 0.70
Accuracy of KNN classifier on test set: 0.59


Cross validation scores

In [182]:
scores = cross_val_score(knn,X_train_scaled,y_train, cv=5)
print("Cross validation scores: {}".format(scores))
print("Average cross-validation scroe: {:.2f}".format(scores.mean()))

Cross validation scores: [ 0.54054054  0.62857143  0.62686567  0.63636364  0.59677419]
Average cross-validation scroe: 0.61


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Confusion Matrix

In [183]:
from sklearn.metrics import confusion_matrix
KNN_best = KNeighborsClassifier(n_neighbors=3).fit(X_train_scaled, y_train)
KNN_predicted = KNN_best.predict(X_test_scaled)
confusion = confusion_matrix(y_test, KNN_predicted)

print('KNN Classification \n', confusion)

KNN Classification 
 [[58  2  0  0  0  1  0  0  0  0  0]
 [ 7  1  0  0  0  0  0  0  0  0  0]
 [ 1  0  4  0  1  0  0  0  0  0  0]
 [ 2  0  0  1  0  0  0  0  0  0  0]
 [ 2  0  0  0  1  0  0  0  1  0  0]
 [ 5  1  0  0  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0  0  0]
 [11  1  0  0  0  0  0  0  1  0  0]
 [ 2  1  0  0  0  0  0  0  0  0  0]
 [ 4  1  0  0  0  1  0  0  0  0  0]]


Classification accuracies report for KNN regression

In [184]:
from sklearn.metrics import classification_report
print(classification_report(y_test, KNN_predicted))

             precision    recall  f1-score   support

          1       0.62      0.95      0.75        61
          2       0.14      0.12      0.13         8
          3       1.00      0.67      0.80         6
          4       1.00      0.33      0.50         3
          5       0.50      0.25      0.33         4
          6       0.00      0.00      0.00         6
          7       0.00      0.00      0.00         2
          9       1.00      1.00      1.00         1
         10       0.50      0.08      0.13        13
         14       0.00      0.00      0.00         3
         16       0.00      0.00      0.00         6

avg / total       0.51      0.59      0.51       113



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Logistic Regression - Finding best params using grid serach and computing cross val scores

In [144]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression()
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }

In [145]:
logit_clf = GridSearchCV(logit, param_grid,cv=5)
logit_clf.fit(X_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

Best parameters

In [146]:
logit_clf.best_params_

{'C': 0.001}

Training and test data accuracies

In [147]:
print('Accuracy of logistic regression on training set: {:.2f}'
     .format(logit_clf.score(X_train, y_train)))
print('Accuracy of logistic regression on test set: {:.2f}'
     .format(logit_clf.score(X_test, y_test)))

Accuracy of logistic regression on training set: 0.86
Accuracy of logistic regression on test set: 0.69


Cross Validation scores

In [148]:
scores = cross_val_score(logit,X_train,y_train, cv=5)
print("Cross validation scores: {}".format(scores))
print("Average cross-validation scroe: {:.2f}".format(scores.mean()))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Cross validation scores: [ 0.55405405  0.62857143  0.67164179  0.60606061  0.5483871 ]
Average cross-validation scroe: 0.60


Confusion Matrix

In [161]:
from sklearn.metrics import confusion_matrix
logit_best = LogisticRegression(C=0.001).fit(X_train, y_train)
logit_predicted = logit_best.predict(X_test)
confusion = confusion_matrix(y_test, logit_predicted)

print('Logistic Regression \n', confusion)

Logistic Regression 
 [[57  2  0  0  0  1  0  0  1  0  0  0]
 [ 5  3  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  5  0  0  0  0  0  0  0  0  0]
 [ 2  0  0  1  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  1  0  0  0  2  0  0  0]
 [ 6  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  0 11  0  0  0]
 [ 2  0  0  0  0  0  0  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]
 [ 4  0  1  0  0  0  0  0  0  0  1  0]]


Classification Accuracies Report for Logit model

In [164]:
from sklearn.metrics import classification_report
print(classification_report(y_test, logit_predicted))

             precision    recall  f1-score   support

          1       0.71      0.93      0.81        61
          2       0.38      0.38      0.38         8
          3       0.83      0.83      0.83         6
          4       1.00      0.33      0.50         3
          5       1.00      0.25      0.40         4
          6       0.00      0.00      0.00         6
          7       0.00      0.00      0.00         2
          9       0.00      0.00      0.00         1
         10       0.79      0.85      0.81        13
         14       0.00      0.00      0.00         3
         15       0.00      0.00      0.00         0
         16       0.00      0.00      0.00         6

avg / total       0.61      0.69      0.63       113



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### Linear SVM - Finding best params using grid serach and computing cross val scores

In [1]:
from sklearn.svm import SVC
LinSVC = SVC(kernel='linear')
LinSVM_params = {'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}

In [2]:
LinSVC_clf = GridSearchCV(LinSVC, LinSVM_params,cv=5)
LinSVC_clf.fit(X_train_scaled,y_train)

NameError: name 'GridSearchCV' is not defined

Best Parameters

In [212]:
LinSVC_clf.best_params_

{'C': 0.01}

Training and test data accuracies

In [221]:
print('Accuracy of linear SVM on training set: {:.2f}'
     .format(LinSVC_clf.score(X_train_scaled, y_train)))
print('Accuracy of linear SVM on test set: {:.2f}'
     .format(LinSVC_clf.score(X_test_scaled, y_test)))

Accuracy of linear SVM on training set: 0.86
Accuracy of linear SVM on test set: 0.66


Cross Validation scores

In [228]:
scores = cross_val_score(SVC(kernel='linear',C=0.01),X_train_scaled,y_train, cv=5)
print("Cross validation scores: {}".format(scores))
print("Average cross-validation score: {:.2f}".format(scores.mean()))

Cross validation scores: [ 0.66216216  0.71428571  0.74626866  0.8030303   0.74193548]
Average cross-validation score: 0.73


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Confusion Matrix 

In [233]:
from sklearn.metrics import confusion_matrix
LinSVC_best = SVC(kernel='linear',C=0.01).fit(X_train_scaled, y_train)
LinSVC_predicted = LinSVC_best.predict(X_test_scaled)
confusion = confusion_matrix(y_test, LinSVC_predicted)

print('Linear SVM \n', confusion)

Linear SVM 
 [[57  2  0  1  0  0  0  0  1  0  0  0]
 [ 6  2  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  5  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  2  0  0  0  0  0  0  0  0]
 [ 2  0  0  0  1  0  0  0  1  0  0  0]
 [ 6  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0  0  0  0]
 [ 6  0  0  0  0  0  0  0  7  0  0  0]
 [ 1  1  0  0  0  0  0  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]
 [ 5  0  0  0  0  0  0  0  0  0  1  0]]


Classification Accuracies Report for Linear SVM

In [234]:
from sklearn.metrics import classification_report
print(classification_report(y_test, LinSVC_predicted))

             precision    recall  f1-score   support

          1       0.66      0.93      0.78        61
          2       0.40      0.25      0.31         8
          3       0.83      0.83      0.83         6
          4       0.67      0.67      0.67         3
          5       1.00      0.25      0.40         4
          6       0.00      0.00      0.00         6
          7       0.00      0.00      0.00         2
          9       1.00      1.00      1.00         1
         10       0.78      0.54      0.64        13
         14       0.00      0.00      0.00         3
         15       0.00      0.00      0.00         0
         16       0.00      0.00      0.00         6

avg / total       0.58      0.66      0.60       113



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### Kernelized SVM - Finding best params using grid serach and computing cross val scores

In [235]:
from sklearn.svm import SVC 
KernelSVC = SVC()
kernelSVC_params = {'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000],'gamma':[1,0.1,0.001,0.0001], 'kernel':['rbf','poly']}

In [236]:
KernelSVC_clf = GridSearchCV(KernelSVC, kernelSVC_params,cv=5)
KernelSVC_clf.fit(X_train_scaled,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.001, 0.0001], 'kernel': ['rbf', 'poly']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

Best Parameters

In [237]:
KernelSVC_clf.best_params_

{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

Training and test data accuracies

In [238]:
print('Accuracy of Kernelized SVM on training set: {:.2f}'
     .format(KernelSVC_clf.score(X_train_scaled, y_train)))
print('Accuracy of Kernelized SVM on test set: {:.2f}'
     .format(KernelSVC_clf.score(X_test_scaled, y_test)))

Accuracy of Kernelized SVM on training set: 0.92
Accuracy of Kernelized SVM on test set: 0.66


Cross Validation scores

In [239]:
scores = cross_val_score(SVC(kernel='rbf',C=10,gamma=0.001),X_train_scaled,y_train, cv=5)
print("Cross validation scores: {}".format(scores))
print("Average cross-validation score: {:.2f}".format(scores.mean()))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Cross validation scores: [ 0.64864865  0.68571429  0.76119403  0.77272727  0.75806452]
Average cross-validation score: 0.73


Confusion Matrix

In [240]:
from sklearn.metrics import confusion_matrix
KernelSVC_best = SVC(kernel='rbf',C=10,gamma=0.001).fit(X_train_scaled, y_train)
KernelSVC_predicted = KernelSVC_best.predict(X_test_scaled)
confusion = confusion_matrix(y_test, KernelSVC_predicted)

print('Linear SVM \n', confusion)

Linear SVM 
 [[57  1  0  1  0  1  0  0  1  0  0  0]
 [ 5  2  0  0  0  0  0  0  0  0  0  1]
 [ 0  1  5  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  2  0  0  0  0  0  0  0  0]
 [ 1  1  0  0  0  0  0  0  1  0  0  1]
 [ 5  0  0  0  0  1  0  0  0  0  0  0]
 [ 1  0  1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0  0  0  0]
 [ 6  0  0  0  0  0  0  0  7  0  0  0]
 [ 1  1  0  0  0  0  0  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]
 [ 3  1  1  0  0  0  0  0  0  0  1  0]]


Classification Accuracies report for Kernelized SVM

In [241]:
from sklearn.metrics import classification_report
print(classification_report(y_test, KernelSVC_predicted))

             precision    recall  f1-score   support

          1       0.71      0.93      0.81        61
          2       0.29      0.25      0.27         8
          3       0.71      0.83      0.77         6
          4       0.67      0.67      0.67         3
          5       0.00      0.00      0.00         4
          6       0.50      0.17      0.25         6
          7       0.00      0.00      0.00         2
          9       1.00      1.00      1.00         1
         10       0.78      0.54      0.64        13
         14       0.00      0.00      0.00         3
         15       0.00      0.00      0.00         0
         16       0.00      0.00      0.00         6

avg / total       0.59      0.66      0.61       113



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
